In [2]:
## Standard libraries
import os
import json
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

/tmp/ipykernel_6405/508501088.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
/home/pmmon/.pyenv/versions/3.9.0/envs/NeuroGS/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


## Define data paths and devices

In [6]:
# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "saved_models/tutorial"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

Global seed set to 42


cuda:0


## Get pre-trained models from web

In [7]:
import urllib.request
from urllib.error import HTTPError
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/"
# Files to download
pretrained_files = ["NodeLevelMLP.ckpt", "NodeLevelGNN.ckpt", "GraphLevelGraphConv.ckpt"]

# Create checkpoint path if it doesn't exist yet
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if "/" in file_name:
        os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

## Define a GCN Layer in PyTorch format

The GCN layer follows the formula: 

$H^{(l+1)} = \sigma\left( \hat{D}^{-1/2} \hat{A} \hat{D}^{-1/2} H^{(l)} W^{(l)} \right)$

Where $W^{(l)}$ is the weight parameters with which we transform the input features into messages $m = H^{(l)} W^{(l)}$, and $\hat{A} = A + I$. The matrix $\hat{D}$ is a diagonal matrix that ensures to take the average. 

In the Pytorch layer, we replace $\hat{D}$ by the simple `sum` operator, and $W^{(l)}$ by a linear layer (that allows for a bias).

In [11]:
class GCNLayer(nn.Module): 
    def __init__(self, c_in, c_out) -> None:
        super().__init__()
        self.projection = nn.Linear(c_in, c_out) # defines the weight matrix

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Assumes to already have added the identity connections.
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        num_neighbors = adj_matrix.sum(dim=-1, keepdims=True)
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)  # batch matrix-matrix product with input (b x n x m) tensor, (b x m x p) tensor, and output a (b x n x p) tensor
        node_feats = node_feats / num_neighbors     # take average
        return node_feats

### Apply an examplary input to layer

In [12]:
# Toy input
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


Initialize GCN-layer with identity weight matrix (input features equal messages)

In [13]:
layer = GCNLayer(c_in=2, c_out=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)



Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1., 2.],
         [3., 4.],
         [4., 5.],
         [4., 5.]]])


As can be seen, the first node's output values are the average of itself and the second node (see first row of output). 
The second node's output values are the average of itself, and all the other nodes' features (see second row of output). 

### Possible problems

As we can see above, nodes 3 and 4 have the same neighbors and, hence, the output features will be the same (as the identity is included). 

In this manner, the GCN layers can make the network forget node-specfic information if we just take the mean over all messages! 

Multiple possible improvements have been proposed. 
- Simples option: using residual connections
- Weight self-connections higher
- Define separate weight matrix for self-connections
- Alternatively, use attention

## Attention

Attention can be applied to graphs, e.g. as it is done by the Graph Attention Network. 

Similarly to the GCN, the graph attention layer creates a message for each node using a linear layer (or weight matrix). 

For attention part: 
- Uses message from node itself as query
- Uses messages to average both as keys and values 
- Score function $f_{attn}$ is implemented as one-layer MLP 

The attention factors are calculated as follows: 

$\alpha_{ij} = \frac{exp(LeakReLU (a[W h_i || W h_j]))}{\sum_{k \in N_i} exp(LeakyReLU ( a[Wh_i || W h_k]))}$

Once all attention factors are obtained, the output features can be computed by performing the weighted average: 

$h_i' = \sigma \left( \sum_{j \in N_i} \alpha_{ij} W h_j \right)$